In [1]:
using RigidBodyDynamics, RigidBodyDynamics.Contact, RigidBodyDynamics.OdeIntegrators
using MomentumBasedControl
using MixedIntegerExperiments
using StaticArrays
using RigidBodyTreeInspector, DrakeVisualizer
using ValkyrieRobot
using BipedControlUtil

In [2]:
val = Valkyrie()
mechanism = val.mechanism;

In [3]:
# add environment
rootframe = root_frame(mechanism)
floor = HalfSpace3D(Point3D(rootframe, 0., 0., 0.), FreeVector3D(rootframe, 0., 0., 1.))
leftwall = HalfSpace3D(Point3D(rootframe, 0., -1., 1.5), FreeVector3D(rootframe, 0., 1., 0.))
rightwall = HalfSpace3D(Point3D(rootframe, 0, 1., 1.5), FreeVector3D(rootframe, 0., -1., 0.))
add_environment_primitive!(mechanism, floor)
add_environment_primitive!(mechanism, leftwall)
add_environment_primitive!(mechanism, rightwall)
;

In [4]:
# Create visualizer
DrakeVisualizer.any_open_windows() || (DrakeVisualizer.new_window(); sleep(1))
vis = Visualizer(parse_urdf(ValkyrieRobot.urdfpath(), mechanism; package_path = [ValkyrieRobot.packagepath()]), :valkyrie);
visualize_environment(mechanism, vis)

In [5]:
remove_fixed_tree_joints!(mechanism)
const state = MechanismState(Float64, mechanism)

MechanismState{Float64, Float64, Float64}(…)

In [6]:
# Initial setup
zero!(state)
kneebend = 1.
for side in instances(Side)
    set_configuration!(state, val.knees[side], [kneebend])
    set_configuration!(state, val.hippitches[side], [-kneebend / 2])
    set_configuration!(state, val.anklepitches[side], [-kneebend / 2])
end
set_configuration!(state, val.floatingjoint, [1; 0; 0; 0; 0; 0; 1.08]) # TODO
settransform!(vis, state);

Point3D in "world": [0.0200832,-0.000967307,1.05451]

In [7]:
const τ = Vector{Float64}(num_velocities(mechanism))
const result = DynamicsResult(Float64, mechanism)
function damped_dynamics!(vd::AbstractArray, sd::AbstractArray, t, state)
    τ[:] = velocity(state)
    scale!(τ, -1.0)
    dynamics!(result, state, τ)
    copy!(vd, result.v̇)
    copy!(sd, result.ṡ)
end

damped_dynamics! (generic function with 1 method)

In [8]:
const controller = MomentumBasedController{Float64}(mechanism);

In [9]:
for body in bodies(mechanism)
    for point in contact_points(body)
        set_contact_regularization!(controller, point, 1e-1)
    end
end
set_joint_accel_regularization!(controller, 0.05)
comgains = PDGains(100., 20.)

MomentumBasedControl.PDGains{Float64}(100.0,20.0)

In [10]:
const result = DynamicsResult(Float64, mechanism)
feet = val.feet
com0 = center_of_mass(state)
function controlled_dynamics!(vd::AbstractArray, sd::AbstractArray, t, state)
    # TODO: extract out: highlevelcontrol
    # Foot accelerations
    T = eltype(controller)
    root = root_body(mechanism)
    for foot in values(feet)
        accel = zero(SpatialAcceleration{T}, default_frame(foot), default_frame(root), default_frame(foot))
        set_desired_accel!(controller, root, foot, accel)
        for point in contact_points(foot)
            set_contact_active!(controller, point, true)
        end
    end
    
    # Centroidal momentum control
    com = center_of_mass(state)
    centroidal = centroidal_frame(controller)
    world_to_centroidal = Transform3D(com.frame, centroidal, -com.v)
    h = transform(momentum(state), world_to_centroidal)
    comerror = FreeVector3D(transform(com0, world_to_centroidal))
    comderror = -FreeVector3D(h.frame, h.linear)
    ldot_desired = comgains.k * comerror + comgains.d * comderror
    hdot_desired = Wrench(ldot_desired, zero(ldot_desired))
    set_desired_momentum_rate!(controller, zero(Wrench{T}, centroidal), eye(SMatrix{6, 6, T}))
    
    τ = control(controller, t, state)
    dynamics!(result, state, τ)
    copy!(vd, result.v̇)
    copy!(sd, result.ṡ)
end

controlled_dynamics! (generic function with 1 method)

In [11]:
# Simulation
# const dyn! = controlled_dynamics!
const dyn! = damped_dynamics!
const integrator = MuntheKaasIntegrator(dyn!, runge_kutta_4(Float64), DrakeVisualizerSink(vis));

In [12]:
controlled_dynamics!(result.v̇, result.ṡ, 0., state);

In [13]:
integrate(integrator, state, 3., 5e-4, maxRealtimeRate = Inf)